<a href="https://colab.research.google.com/github/marco-siino/text_preprocessing_impact/blob/main/FNS_DS/RoBERTa_FNS_TextPreProImpact_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Text preprocessing worth the time: A comparative survey on the impact of common techniques on NLP model performances. 
- - - 
RoBERTa ON FNS DS EXPERIMENTS NOTEBOOK 
- - -
RoBERTa on Fake News Spreaders Dataset.
Code by M. Siino. 

From the paper: "Text preprocessing worth the time: A comparative survey on the impact of common techniques on NLP model performances." by M.Siino et al.



## Importing modules.

In [ ]:
!pip install simpletransformers
!pip install tensorboardx

import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np
import torch
import nltk
import pandas as pd

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from keras.models import Model
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
from textblob import TextBlob
nltk.download('stopwords')
nltk.download('punkt')
from io import open
from pathlib import Path
from simpletransformers.classification import ClassificationModel, ClassificationArgs


C:\Users\Domenico\.conda\envs\tf\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Domenico\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Domenico\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]


## Importing DS and extract in current working directory.

In [ ]:
# Url obtained starting from this: https://drive.google.com/file/d/19ZcqEv88euKB71HfAWjTGN3uCKp2qsfP/ and forcing export=download.
urlTrainingSet = "https://github.com/marco-siino/fake_news_spreaders_detection/raw/main/dataset/pan20-author-profiling-training-2020-02-23.zip"
urlTestSet="https://github.com/marco-siino/fake_news_spreaders_detection/raw/main/dataset/pan20-author-profiling-test-2020-02-23.zip"

training_set = tf.keras.utils.get_file("pan20-author-profiling-training-2020-02-23.zip", urlTrainingSet,
                                    extract=True, archive_format='zip',cache_dir='.',
                                    cache_subdir='')
test_set = tf.keras.utils.get_file("pan20-author-profiling-test-2020-02-23.zip", urlTestSet,
                                    extract=True, archive_format='zip',cache_dir='.',
                                    cache_subdir='')

training_set_dir = os.path.join(os.path.dirname(training_set), 'pan20-author-profiling-training-2020-02-23')
test_set_dir = os.path.join(os.path.dirname(test_set), 'pan20-author-profiling-test-2020-02-23')

print(training_set)
print(training_set_dir)

!ls -A

.\pan20-author-profiling-training-2020-02-23.zip
.\pan20-author-profiling-training-2020-02-23


'ls' is not recognized as an internal or external command,
operable program or batch file.


## Build folders hierarchy to use Keras folders preprocessing function.

In [ ]:
### Training Folders. ###

# First level directory.
if not os.path.exists('train_dir_en'):
    os.makedirs('train_dir_en')

# Class labels directory.
if not os.path.exists('train_dir_en/0'):
    os.makedirs('train_dir_en/0')
if not os.path.exists('train_dir_en/1'):
    os.makedirs('train_dir_en/1')

# Make Py variables.
train_dir='train_dir_'

## Test Folders. ##
# First level directory.
if not os.path.exists('test_dir_en'):
    os.makedirs('test_dir_en')

# Class labels directory.
if not os.path.exists('test_dir_en/0'):
    os.makedirs('test_dir_en/0')
if not os.path.exists('test_dir_en/1'):
    os.makedirs('test_dir_en/1')

# Make Py variables.
test_dir='test_dir_'

!ls -A

'ls' is not recognized as an internal or external command,
operable program or batch file.


## Set language and directory paths.


In [ ]:
# Set en and es ground truth file path for train_dir. We haven't a ground truth file for the test set.
language='en'

truth_file_training_dir_en=training_set_dir+'/'+language+'/'
truth_file_training_path_en = truth_file_training_dir_en+'truth.txt'

truth_file_test_dir=test_set_dir
truth_file_test_path_en = truth_file_test_dir+'/'+language+'.txt'

## Read truth.txt to organize training and test dataset folders.

In [ ]:
# Open the file truth.txt with read only permit.
f = open(truth_file_training_path_en, "r")
# use readline() to read the first line 
line = f.readline()
# use the read line to read further.
# If the file is not empty keep reading one line
# at a time, till the file is empty
while line:
    # Split line at :::
    x = line.split(":::")
    fNameXml = x[0]+'.xml'
    fNameTxt = x[0]+'.txt'
    # Second coord [0] gets just the first character (label) and not /n too.
    label = x[1][0]

    # Now move the file to the right folder.
    if os.path.exists(truth_file_training_dir_en+fNameXml):
      os.rename(truth_file_training_dir_en+fNameXml, './train_dir_'+language+'/'+label+'/'+fNameTxt )

    # use readline() to read next line
    line = f.readline()

# Open the file truth.txt with read only permit.
f = open(truth_file_test_path_en, "r")
# use readline() to read the first line 
line = f.readline()
# use the read line to read further.
# If the file is not empty keep reading one line
# at a time, till the file is empty
while line:
    # Split line at :::
    x = line.split(":::")
    fNameXml = x[0]+'.xml'
    fNameTxt = x[0]+'.txt'
    # Second coord [0] gets just the first character (label) and not /n too.
    label = x[1][0]

    # Now move the file to the right folder.
    if os.path.exists(truth_file_test_dir+'/'+language+'/'+fNameXml):
      os.rename(truth_file_test_dir+'/'+language+'/'+fNameXml, './test_dir_'+language+'/'+label+'/'+fNameTxt )

    # use readline() to read next line
    line = f.readline()

FileExistsError: ignored

## Building the dataset.

In [ ]:
# Generate full randomized training set.
batch_size=1

en_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir+language, 
    batch_size=batch_size,
    shuffle=False
    )

en_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    test_dir+language, 
    batch_size=batch_size,
    shuffle=False
    )

train_ds=en_train_ds.shuffle(300,seed=1, reshuffle_each_iteration=False)
test_ds=en_test_ds.shuffle(200,seed=1, reshuffle_each_iteration=False)

train_ds_size=len(train_ds)
test_ds_size=len(test_ds)

Found 300 files belonging to 2 classes.
Found 200 files belonging to 2 classes.


## Functions to pre-process source text. (A detailed discussion on our paper)

In [ ]:
# Do-Nothing preprocessing function.
def DON(input_data):
  tag_open_CDATA_removed = tf.strings.regex_replace(input_data, '<\!\[CDATA\[', ' ')
  tag_closed_CDATA_removed = tf.strings.regex_replace(tag_open_CDATA_removed,'\]{1,}>', ' ')
  tag_author_lang_en_removed = tf.strings.regex_replace(tag_closed_CDATA_removed,'<author lang="en">', ' ')
  tag_closed_author_removed = tf.strings.regex_replace(tag_author_lang_en_removed,'</author>', ' ')
  tag_open_documents_removed = tf.strings.regex_replace(tag_closed_author_removed,'<documents>\n(\t){0,2}', '')
  output_data = tf.strings.regex_replace(tag_open_documents_removed,'</documents>\n(\t){0,2}', ' ')
  return output_data

# Lowercasing preprocessing function.
def LOW(input_data):  
  return tf.strings.lower(DON(input_data))

# Removing Stop Words function.
def RSW(input_data):
  output_data = DON(input_data)

  #print("\n\nInput data è il seguente tensore:")
  #print(output_data)

  #print("Lo converto in stringa e diventa:")
  # Il seguente try per l'adattamento del ts. Nell'except caso della simulazione vera e propria.
  try:
    input_string=output_data[0]

  # # # # # # # Questo è il caso della chiamata a funzione per la simulazione vera e propria.  
  except:
    #print("\n\n****CASO DELLA SIMULAZIONE VERA E PROPRIA****\n\n")
    #print("\nQuesto è il contenuto di output data in caso di simulazione")
    #print(output_data)
    input_string=output_data
    
    try:
      input_string = input_string.numpy()
    
    except:
      #print("This one is not a tensor!")
      return output_data

    else:
      #print("\nEstraendo il contenuto del tensore risulta:")
      input_string=(str(input_string))[2:-1]

    #print(input_string)
    blob = TextBlob(str(input_string)).words

    outputlist = [word for word in blob if word not in stopwords.words('english')]
    #print("tolte le stopword inglesi diventa:")

    output_string = (' '.join(word for word in outputlist))
    #print(output_string)  

    output_tensor=tf.constant(output_string)
    #print(output_tensor)

    return output_tensor

   # # # # # # # Questo è il caso dell'adattamento del TS.   
  else:
    
    try:

      # input_string = input_string.numpy() [0]
      input_string = input_string.numpy()
    
    except:
      #print("This one is not a tensor!")
      return output_data

    else:
      input_string=(str(input_string))[2:-1]

    #print(input_string)
    blob = TextBlob(str(input_string)).words

    outputlist = [word for word in blob if word not in stopwords.words('english')]
    #print("Tolte le stopword inglesi diventa:")

    output_string = (' '.join(word for word in outputlist))
    #print(output_string)  

    output_tensor=tf.constant([[output_string]])
    #print(output_tensor)

    return output_tensor

  return output_data

# Porter Stemmer preprocessing function.
def STM(input_data):
  output_data = DON(input_data)
  stemmer = PorterStemmer()

  #print("\n\nInput data è il seguente tensore:")
  #print(output_data)

  #print("Lo converto in stringa e diventa:")
  # Il seguente try per l'adattamento del ts. Nell'except caso della simulazione vera e propria.
  try:
    input_string=output_data[0]

  # # # # # # # Questo è il caso della chiamata a funzione per la simulazione vera e propria.  
  except:
    #print("\n\n****CASO DELLA SIMULAZIONE VERA E PROPRIA****\n\n")
    #print("\nQuesto è il contenuto di output data in caso di simulazione")
    #print(output_data)
    input_string=output_data
    
    try:
      input_string = input_string.numpy()
    
    except:
      #print("This one is not a tensor!")
      return output_data

    else:
      #print("\nEstraendo il contenuto del tensore risulta:")
      #print(input_string)
      input_string=(str(input_string))[2:-1]

    #print(input_string)
    blob = TextBlob(str(input_string)).words

    outputlist = [stemmer.stem(word) for word in blob]

    output_string = (' '.join(word for word in outputlist))
    #print(output_string)  

    output_tensor=tf.constant(output_string)
    #print(output_tensor)

    return output_tensor

   # # # # # # # Questo è il caso dell'adattamento del TS.   
  else:
    
    try:
      #input_string = input_string.numpy()[0]
      input_string = input_string.numpy()
      #print(input_string)
    
    except:
      #print("This one is not a tensor!")
      return output_data

    else:
      input_string=(str(input_string))[2:-1]

    #print(input_string)
    blob = TextBlob(str(input_string)).words

    outputlist = [stemmer.stem(word) for word in blob]

    output_string = (' '.join(word for word in outputlist))

    output_tensor=tf.constant([[output_string]])
    #print(output_tensor)

    return output_tensor

  return output_data

## Define the combined preprocessing functions. (The base functions are: DON, LOW, RSW and STM).

In [ ]:
## SECTION WITH PAIRS OF PREPRO FUNCTIONS. APPLICATION ORDER MATTERS (...IN FOLLOWING SECTIONS TOO).
#...5
def LOW_RSW(input_data):
  return RSW(LOW(input_data))

# 6
def LOW_STM(input_data):
  return STM(LOW(input_data))

# 7
def RSW_LOW(input_data):
  return LOW(RSW(input_data))

# 8
def RSW_STM(input_data):
  return STM(RSW(input_data))

# 9
def STM_LOW(input_data):
  return LOW(STM(input_data))

# 10
def STM_RSW(input_data):
  return RSW(STM(input_data))
  
# 11
def LOW_STM_RSW(input_data):
  return RSW(STM(LOW(input_data)))

# 12
def LOW_RSW_STM(input_data):
  return STM(RSW(LOW(input_data)))

# 13
def STM_LOW_RSW(input_data):
  return RSW(LOW(STM(input_data)))

# 14
def STM_RSW_LOW(input_data):
  return LOW(RSW(STM(input_data)))

# 15
def RSW_LOW_STM(input_data):
  return STM(LOW(RSW(input_data)))

# 16
def RSW_STM_LOW(input_data):
  return LOW(STM(RSW(input_data)))

## Define a dictionary with -> function_names:prepro_function_caller. And a dictionary to store model results.

In [ ]:
model_results = {}
prepro_functions_dict_base = {
    'DON':DON,
    'LOW':LOW,
    'RSW':RSW,
    'STM':STM
    }

# 3 prepro functions = 15 combs...+1 for do_nothing

prepro_functions_dict_comb = {
    # 1. Do nothing 
    'DON': DON,
    # 2. Lowercasing 
    'LOW':LOW,
    # 3. Removing Stopwords
    'RSW':RSW, 
    # 4. Porter Stemming
    'STM':STM,
    # 5. LOW->RSW
    'LOW_RSW':LOW_RSW, 
    # 6. LOW->STM
    'LOW_STM':LOW_STM,
    # 7. RSW->LOW
    'RSW_LOW':RSW_LOW,
    # 8. RSW->STM
    'RSW_STM':RSW_STM,
    # 9. STM->LOW
    'STM_LOW':STM_LOW,
    # 10. STM->RSW
    'STM_RSW':STM_RSW,
    # 11. LOW->STM->RSW
    'LOW_STM_RSW':LOW_STM_RSW,  
    # 12. LOW->RSW->STM
    'LOW_RSW_STM':LOW_RSW_STM,
    # 13. STM->LOW->RSW
    'STM_LOW_RSW':STM_LOW_RSW,
    # 14. STM->RSW->LOW
    'STM_RSW_LOW':STM_RSW_LOW,
    # 15. RSW->LOW->STM
    'RSW_LOW_STM':RSW_LOW_STM,
    # 16. RSW->STM->LOW
    'RSW_STM_LOW':RSW_STM_LOW
}

for key in prepro_functions_dict_comb:
  print(key)
  model_results[key]=[]

DON
LOW
RSW
STM
LOW_RSW
LOW_STM
RSW_LOW
RSW_STM
STM_LOW
STM_RSW
LOW_STM_RSW
LOW_RSW_STM
STM_LOW_RSW
STM_RSW_LOW
RSW_LOW_STM
RSW_STM_LOW


## Function to convert DSs to Pandas Dataframe

In [ ]:
def preprocess_and_convert_ds(preprocessing_function):
  # Convert English dataset.
  train_df = [] # will contain text and label
  for element in train_ds:
    authorDocument=element[0]
    label=int(element[1].numpy())
    #print(authorDocument[0])
    text = preprocessing_function(authorDocument[0].numpy()).numpy().decode('UTF-8')
    train_df.append({
        'text':text,
        'label':label
    })
  train_df = pd.DataFrame(train_df)

  test_df = [] # will contain text and label
  for element in test_ds:
    authorDocument=element[0]
    label=int(element[1].numpy())
    #print(authorDocument[0])
    text = preprocessing_function(authorDocument[0].numpy()).numpy().decode('UTF-8')
    test_df.append({
        'text':text,
        'label':label
    })
  test_df = pd.DataFrame(test_df)

  return train_df, test_df


## Print some RAW and preprocessed samples (No need to execute)

In [ ]:
for idx, element in enumerate(raw_train_ds_es):
  if idx>1: break
  authorDocument=element[0]
  label=element[1]
  temp = custom_standardization(authorDocument[0].numpy()).numpy().decode('UTF-8')
  print("Not-Preprocessed samples: \n",authorDocument)
  print("Preprocessed samples: \n",temp)

NameError: ignored

## Some parameters definition...

In [ ]:
# check gpu
cuda_available = torch.cuda.is_available()

print('Cuda available? ',cuda_available)

num_epochs_per_run = 10
num_runs = 5

Cuda available?  False


## Training and evaluation of the model

In [ ]:
for key in prepro_functions_dict_comb:
  model_results[key]=[]

for key in prepro_functions_dict_comb:

  model_args = ClassificationArgs(num_train_epochs=1, 
                                      no_save=True, 
                                      no_cache=True, 
                                      overwrite_output_dir=True)

  model = ClassificationModel("roberta", 
                                  'roberta-base', 
                                  args = model_args, 
                                  num_labels=2, 
                                  use_cuda=cuda_available)

  runs_accuracy = []

  print("\n\n* * * * EVALUATION USING", key, "AS PREPROCESSING FUNCTION * * * *")

  # Preprocess train and test set and convert to DFs.
  train_df,test_df = preprocess_and_convert_ds(prepro_functions_dict_comb[key])
  
  for run in range(1,(num_runs+1)):
    epochs_accuracy=[]
    model = ClassificationModel("roberta", 
                                    'roberta-base', 
                                    args = model_args, 
                                    num_labels=2, 
                                    use_cuda=cuda_available)
    for epoch in range (0,num_epochs_per_run):
      print("\nEPOCH NUMBER: ", epoch)
      # train model
      print("\nNOW TRAIN THE MODEL.")
      model.train_model(train_df,show_running_loss=False)
      print("\nNOW EVALUATE THE TEST DF.")
      result, model_outputs, wrong_predictions = model.eval_model(test_df)
      # Results on test set.
      print(result)
      correct_predictions = result['tp']+result['tn']
      print("Correct predictions are: ",correct_predictions)
      total_predictions = result['tp']+result['tn']+result['fp']+result['fn']
      print("Total predictions are: ",total_predictions)
      accuracy = correct_predictions/total_predictions
      print("Accuracy on test set is:",accuracy,"\n\n")
      epochs_accuracy.append(accuracy)

    print(epochs_accuracy)
    runs_accuracy.append(max(epochs_accuracy))

  runs_accuracy.sort()
  print("\n\n Over all runs maximum accuracies are:", runs_accuracy)
  print("The median is:",runs_accuracy[2])

  if (runs_accuracy[2]-runs_accuracy[0])>(runs_accuracy[4]-runs_accuracy[2]):
    max_range_from_median = runs_accuracy[2]-runs_accuracy[0]
  else:
    max_range_from_median = runs_accuracy[4]-runs_accuracy[2]
  final_result = str(runs_accuracy[2])+" +/- "+ str(max_range_from_median)
  model_results[key].append(final_result)
  print("RoBERTa Accuracy Score on Test set -> ",model_results[key])


## Now show compact results in a table.

In [ ]:
print(" PREPRO FUNCTION    |  Test Accuracy   |",end = '')

print("\n")
for prepro_func in prepro_functions_dict_comb:
  #print(prepro_func,"\t\t\t",format(round(model_results[prepro_func][0],4),'.4f'),"\t\t",end='')
  result = model_results[prepro_func][0]
  # result = format(round(model_results[prepro_func][0],4),'.4f')
  print(f'{prepro_func:27}{ result :12}')
  print("\n")

 PREPRO FUNCTION    |  Test Accuracy   |

DON                        0.695 +/- 0.019999999999999907


LOW                        0.655 +/- 0.04500000000000004


RSW                        0.705 +/- 0.015000000000000013


STM                        0.66 +/- 0.030000000000000027


LOW_RSW                    0.665 +/- 0.020000000000000018


LOW_STM                    0.625 +/- 0.040000000000000036


RSW_LOW                    0.67 +/- 0.020000000000000018


RSW_STM                    0.65 +/- 0.15000000000000002


STM_LOW                    0.66 +/- 0.135


STM_RSW                    0.66 +/- 0.15500000000000003


LOW_STM_RSW                0.64 +/- 0.10499999999999998


LOW_RSW_STM                0.645 +/- 0.010000000000000009


STM_LOW_RSW                0.64 +/- 0.14


STM_RSW_LOW                0.64 +/- 0.14


RSW_LOW_STM                0.645 +/- 0.125


RSW_STM_LOW                0.64 +/- 0.015000000000000013


